# Assignment NMT: Intro to NLP

The aim of this homework is to familiarize you with sequence-to-sequence language modeling, specifically using an encoder-decoder model. In this notebook, you are provided with pre-written code for a simple sequence-to-sequence model that already works and learns how to reverse short sequences of numbers.

If you run this whole jupyter notebook, it will learn to reverse short sequences of numbers. Although much of this code you will not be modifying, we recommend reading through it to get a sense of how the model and training works.

This starter code is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website and the COMS course at Columbia University by Professor Kathy McKeown. 

### Overview

Your assignment is to:
 1. adapt this notebook to work on the English-Italian language pair from Tataoeba website
 2. Implement a beam search function
 

Write all your code **in this jupyter notebook**. Cells are provided where you should be implementing your code. 

You do not need to modify any code to train the model. You may modify the `trainIters` function, if you would like to improve how you track progress, or change parameters while training. For example, it can be useful to decrease the teacher-forcing ratio as training progresses.


I would recommend you run this notebook as it is, first. You should be able to run it with the dummy data provided without making ANY modifications (except the cell where the data are loaded). Then, start making your changes as requested.



In [1]:
# You may modify this cell if you like

import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# DO NO MODIFY

# this is useful for checking if your code is successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cuda')

In [3]:
# DO NOT MODIFY

SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 5000

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### 1. Load the data (10 points)

Load in the en-it data from http://www.manythings.org/anki/ita-eng.zip

 similar to how the dummy number reversal dataset is loaded above. That is, use the same `torchtext.data.Field` and `torchtext.data.TabularDataset` classes.

In [4]:
# WRITE YOUR CODE FOR LOADING THE ITALIAN<->ENGLISH DATA HERE
#TODO 
#create 6 files, English|Italian data for train|dev|test, one sentence per line.
#you can ignore this cell when you are running the code the first time through on the dummy reversal dataset

# Import from external file
X_train = pd.read_csv("output/x_train.csv")
X_test = pd.read_csv("output/x_test.csv")

# Set paths
train_path = 'output/x_train_nn.csv'
test_path = 'output/x_test_nn.csv'

# Save files
X_train[['question', 'prep_answer', 'cluster']].to_csv(train_path, index=False, header=False)
X_test[['question', 'prep_answer', 'cluster']].to_csv(test_path, index=False, header=False)

# Create pytorch variables
src = torchtext.data.Field(
    batch_first=True,
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True,
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_test = torchtext.data.TabularDataset(
        path=test_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

Have a look at the vocab and some example data points.

*If you have loaded in the data correctly, the code in the cell below should work without any modification.*

In [5]:
# You may modify this cell if you like

src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

#showing output below for toy dataset

20 tokens from input vocab:
 ['<unk>', '<pad>', 'the', 'to', 'of', 'a', 'I', 'and', 'in', 'is', 'that', 'for', 'be', 'have', 'it', 'this', 'are', 'on', 'with', 'my']

20 tokens from output vocab:
 ['<unk>', '<pad>', 'the', 'to', 'of', 'and', 'a', 'is', 'in', 'that', 'for', 'you', 'not', 'are', 'be', 'it', 'or', 'with', 'have', 'as']

num training examples: 1034

example train data:
src:
 ['What', 'about', 'my', 'summer', 'vacation?', 'Should', 'I', 'cancel', 'that?']
tgt:
 ['<sos>', 'no', 'one', 'really', 'knows', 'what', 'the', 'course', 'of', 'the', 'outbreak', 'will', 'be.', 'some', 'airlines', 'have', 'already', 'announced', 'cuts', 'through', 'may,', 'and', 'president', 'trump', 'has', 'mentioned', 'isolation', 'measures', 'lasting', 'through', 'july', 'or', 'august.', 'the', 'prevailing', 'optimistic', 'guess', 'among', 'experts', 'for', 'when', 'the', 'outbreak', 'will', 'abate', 'hovers', 'at', 'about', 'two', 'months.', '<eos>']


### Model definition and training functions

In [6]:
# DO NOT MODIFY

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [7]:
# DO NOT MODIFY

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [8]:
# You may modify this cell

def trainIters(encoder, decoder, n_iters, print_every=10000, learning_rate=0.04, teacher_forcing_ratio=0.2):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

###  Create and train a model

In [9]:
# You may modify this cell

hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.



In [10]:
# You may modify this cell
# but be sure that it prints some indication of how training is progressing

trainIters(encoder1, decoder1, 1, print_every=10000)

Running 10 epochs...
End of epoch 0, avg loss 375.16
End of epoch 1, avg loss 438.55
End of epoch 2, avg loss 528.41
End of epoch 3, avg loss 493.59
End of epoch 4, avg loss 561.61
End of epoch 5, avg loss 521.60
End of epoch 6, avg loss 498.38
End of epoch 7, avg loss 466.56
End of epoch 8, avg loss 505.29
End of epoch 9, avg loss 519.03


In [11]:
# DO NOT MODIFY

def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

Have a look at some generated sequences! This is the fun part.

In [12]:
# You may modify this cell

# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['Can', 'I', 'step', 'out', 'to', 'withdraw', 'money', 'from', 'the', 'ATM?']
national the reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported which reported 

### Implement beam search (10 points for ITCS 4111 students, 15 points for the ITCS 5111/DSBA 6010 students)

We provide an evaluation function that performs greedy decoding on any input sequence provided. 

(ITCS 4111 students) You must write a new  function that implements beam search for a beam size = 2. 

(ITCS 5111/DSBA 6010 students) You must write a new  function that implements beam search for an arbitrary beam size. Let the beam size be an input parameter to your function. 

In greedy decoding, at each decoding step the most likely word is selected, resulting in one decoded sequence output. 

In beam search, at each decoding step the top k most
likely sequences are selected. Each of these k sequences is then used to generate the next step; the top k next words per sequence are considered (for a total of k ∗ k sequences)
and the top k sequences are selected to take to the next decoding step. At the end, you have k decoded sequence outputs.


In [13]:
# WRITE YOUR CODE FOR THE BEAM SEARCH HERE


#there are several implementation of beam search on the web. You can adapt those for this assignment. 
#Please provide proper credit to the original source if you are doing so
#one example is here: https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py

### The following code has been adapted from https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py
import operator
from queue import PriorityQueue

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, embedding_size=7, dropout=0.1):
        '''
        Illustrative decoder
        '''
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(num_embeddings=output_size,
                                      embedding_dim=embedding_size,
                                      )

        self.rnn = nn.GRU(embedding_size, hidden_size, dropout=dropout, batch_first=False)
        self.dropout_rate = dropout
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).transpose(0, 1)  # [B,1] -> [ 1, B, D]
        embedded = F.dropout(embedded, self.dropout_rate)

        output = embedded

        output, hidden = self.rnn(output, hidden)

        out = self.out(output.squeeze(0))
        output = F.log_softmax(out, dim=1)
        return output, hidden

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward


def beam_decode(target_tensor, decoder_hiddens, beam_width = 10):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    topk = 1  # how many sentence do you want to generate
    decoded_batch = []

    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
        if isinstance(decoder_hiddens, tuple):  # LSTM case
            decoder_hidden = (decoder_hiddens[0][:,idx, :].unsqueeze(0),decoder_hiddens[1][:,idx, :].unsqueeze(0))
        else:
            decoder_hidden = decoder_hiddens[:, idx, :].unsqueeze(0)
        #encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

        # Start with the start of the sentence token
        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_TOKEN and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch

#you may need to add a helper function to see if your implementation of beam search works
#add that helper function below

def gen_tensors(tab_data, n_iters=1):

# note batch size of 1, just for simplicity
    batch_iterator = torchtext.data.Iterator(
        dataset=tab_data, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        for batch in batch_generator:
            step += 1

# get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            #target_tensor = getattr(batch, 'tgt')

 # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            #target_tensor = target_tensor[0]

# reshape input and target tensors
            #input_tensor.unsqueeze_(-1)
            #input_tensor = input_tensor.expand(1,input_tensor.shape[0],1)
            #input_tensor = torch.transpose(input_tensor, 1, 2)
            target_tensor = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)
            target_tensor = target_tensor.expand(1, target_tensor.shape[0])

 # get an initial hidden state for the encoder
    encoder_hidden = encoder1.initHidden()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(MAX_LEN, encoder1.hidden_size, device=mydevice)

    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder1(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    return encoder_hidden, target_tensor

In [14]:
decoder_hidden, target_tensor = gen_tensors(data_train)

In [15]:
import gc
gc.collect()
decoded = beam_decode(target_tensor, decoder_hidden, beam_width = 2)

d:\programdata\venv\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


TypeError: '<' not supported between instances of 'BeamSearchNode' and 'BeamSearchNode'

In [ ]:
decoded[0][0][0]

In [ ]:
decoded_words = []
for di in range(decoded.shape[2]):
            topi = decoded[0][0][di]
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

In [ ]:
decoded_words

In [ ]:
mydevice = 'cpu'

In [ ]:
mydevice = 'cuda'

In [ ]:
import gc
gc.get_count()